In [12]:
# analyse label cnt
import json
import pandas as pd

d = json.loads(open('../data/train_data.json').read())
df_data = []
for k in d:
    df_data.append([k, len(d[k])])
print(df_data[:5])

[['w_e069b6a', 2], ['w_689e1cd', 2], ['w_dd677ef', 1], ['w_1b70f3f', 1], ['w_efa100e', 6]]


In [13]:
df = pd.DataFrame(df_data, columns=['l', 'cnt'])
df.head()

,l,cnt
0,w_e069b6a,2
1,w_689e1cd,2
2,w_dd677ef,1
3,w_1b70f3f,1
4,w_efa100e,6


In [14]:
top5_labels = df.sort_values('cnt').l.values[-5:]
print(top5_labels)

['w_0369a5c' 'w_9c506f6' 'w_9b5109b' 'w_23a388d' 'new_whale']


In [15]:
new_df = df.sort_values('cnt')
print(len(new_df[new_df.cnt>10]))
new_df.tail(20)

228


,l,cnt
889,w_60ce6fc,37
571,w_5a2634c,37
695,w_a9304b9,37
2897,w_778e474,40
3523,w_5e8e218,40
1577,w_343f088,40
3069,w_f0fe284,45
1250,w_789c969,45
483,w_d405854,47
3837,w_2b069ba,48


In [16]:
df.describe()

,cnt
count,5005.000000
mean,5.067133
std,136.634756
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,9664.000000


In [17]:
for i in [2, 3, 4]:
    print(i, len(df[df.cnt < i]))

2 2073
3 3358
4 3926


In [18]:
# analyse public sub
pub_df = pd.read_csv('../data/pub_864.csv')
pub_df.head(10)

,Image,Id
0,00028a005.jpg,w_dec7ffd new_whale w_ab44ae4 w_f009245 w_2a0e...
1,000dcf7d8.jpg,w_4132bb8 new_whale w_3de579a w_9d29561 w_9119...
2,000e7c7df.jpg,w_94c3bf0 new_whale w_3de579a w_fc6dd24 w_d7ae...
3,0019c34f4.jpg,w_a335fc2 w_c99807e new_whale w_59052ad w_0135...
4,001a4d292.jpg,new_whale w_20eeb29 w_5b40032 w_ce66f9a w_82e7...
5,00247bc36.jpg,new_whale w_d7aef56 w_ff5fc91 w_2284d86 w_3137...
6,0027089a4.jpg,w_7e2eb3d w_2284d86 new_whale w_a9304b9 w_0090...
7,002de4d94.jpg,new_whale w_9119b08 w_c7a611a w_a9304b9 w_0d9d...
8,002f52f0c.jpg,new_whale w_53feb91 w_3879fa0 w_02545ea w_d7ae...
9,002fd89d4.jpg,new_whale w_261878d w_eae983e w_d6907f5 w_38c0...


In [19]:
preds = pub_df.Id.values
preds = [p.split(' ') for p in preds]
print(preds[0])


# save top3 for filter speed up
for i in range(5):
    print('top', i)
    tmp_set = set()
    for p in preds:
        for element in p[:i+1]:
            tmp_set.add(element)
    if i==2:
        with open('../data/top3_filter.json','w') as fout:
            json.dump(list(tmp_set), fout, indent=2)
            print('save done')
    print(len(tmp_set))
    print('----------')

['w_dec7ffd', 'new_whale', 'w_ab44ae4', 'w_f009245', 'w_2a0e817', 'w_700ebb4']
top 0
3266
----------
top 1
4021
----------
top 2
save done
4463
----------
top 3
4698
----------
top 4
4827
----------


In [20]:
# simple test, replace last 2 with top res in top5
def update_p(p_list, top5):
    tmp_p = p_list[:3]
    tmp_filter_top = [l for l in top5 if l not in tmp_p]
    tmp_p = tmp_p + tmp_filter_top[-2:]
    return tmp_p


# test
print(update_p(preds[0], top5_labels))

test_new_pred = [' '.join(update_p(p, top5_labels)) for p in preds]
new_df = pub_df.copy()
new_df.Id = test_new_pred
print(new_df.head(10))
new_df.to_csv('../submissions/replace_2.csv',index=False)  # score 860, target

['w_dec7ffd', 'new_whale', 'w_ab44ae4', 'w_9b5109b', 'w_23a388d']
           Image                                                 Id
0  00028a005.jpg  w_dec7ffd new_whale w_ab44ae4 w_9b5109b w_23a388d
1  000dcf7d8.jpg  w_4132bb8 new_whale w_3de579a w_9b5109b w_23a388d
2  000e7c7df.jpg  w_94c3bf0 new_whale w_3de579a w_9b5109b w_23a388d
3  0019c34f4.jpg  w_a335fc2 w_c99807e new_whale w_9b5109b w_23a388d
4  001a4d292.jpg  new_whale w_20eeb29 w_5b40032 w_9b5109b w_23a388d
5  00247bc36.jpg  new_whale w_d7aef56 w_ff5fc91 w_9b5109b w_23a388d
6  0027089a4.jpg  w_7e2eb3d w_2284d86 new_whale w_9b5109b w_23a388d
7  002de4d94.jpg  new_whale w_9119b08 w_c7a611a w_9b5109b w_23a388d
8  002f52f0c.jpg  new_whale w_53feb91 w_3879fa0 w_9b5109b w_23a388d
9  002fd89d4.jpg  new_whale w_261878d w_eae983e w_9b5109b w_23a388d
